### Notebook: mcmc_k_equals_c_beta_over_alpha.ipynb

**Objective:** Test the hypothesis that the observed decay constant k equals c × (β/α) where c ≈ 1 is a normalization factor. Use MCMC to fit MIDIS data and extract c.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import emcee
from scipy.optimize import minimize

# parameters (papermill-friendly)
DATA_CSV = "data/midis_f560w_masslim.csv"

# Load MIDIS data from SST source
midis_data = pd.read_csv(DATA_CSV)
print("MIDIS observational data:")
print(midis_data)

# QH parameters
alpha = 0.314
beta = 0.0158
beta_over_alpha = beta / alpha

# Define model
def model(z, c, gamma_0):
    """Exponential decay model with normalization c"""
    k_pred = c * beta_over_alpha
    return gamma_0 * np.exp(-k_pred * z)

# Likelihood function
def log_likelihood(theta, z, gamma_obs, gamma_err):
    c, gamma_0 = theta
    gamma_pred = model(z, c, gamma_0)
    chi2 = np.sum((gamma_obs - gamma_pred)**2 / gamma_err**2)
    return -0.5 * chi2

# MCMC analysis
def log_prior(theta):
    c, gamma_0 = theta
    if 0.5 < c < 2.0 and 10.0 < gamma_0 < 100.0:
        return 0.0
    return -np.inf

def log_probability(theta, z, gamma_obs, gamma_err):
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, z, gamma_obs, gamma_err)

# Initial guess
theta_init = [1.0, 50.0]
print(f"\\nTesting c = {theta_init[0]:.1f} (should be ≈ 1.0)")
print(f"β/α = {beta_over_alpha:.4f}")
print(f"Predicted k = c × β/α = {theta_init[0] * beta_over_alpha:.4f}")
